In [1]:
import andi
import numpy as np
import csv as csv

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
#from keras.models import load_model
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
#from data_split import data_split
from utils import data_norm, data_reshape, many_net_uhd, my_atan

Using TensorFlow backend.


Generating trajectories to test the data. 3 trajectories for each dimension of inference and classification tasks. This requires the AnDi package, downloadable at https://github.com/AnDiChallenge

In [2]:
AD = andi.andi_datasets()
i=200

X1, Y1, X2, Y2, X3, Y3 = AD.andi_dataset(N = 3, tasks = [1,2,3], dimensions = [1,2,3],
                                        min_T = i, max_T = i+1, )

Creating a dataset for task(s) [1, 2, 3] and dimension(s) [1, 2, 3].
Generating dataset for dimension 1.


/opt/anaconda3/lib/python3.7/site-packages/fbm/fbm.py:172: UserWarning: Combination of increments n and Hurst value H invalid for Davies-Harte method. Reverting to Hosking method. Occurs when n is small and Hurst is close to 1. 
  "Combination of increments n and Hurst value H "


Generating dataset for dimension 2.
Generating dataset for dimension 3.


# Inference of 1d trajectories

The output of the inference networks is the inferred anomalous exponent for each trajectory. Trajectories must be of equal length

Importing the networks used for inference in 1d

In [3]:
centers_inf_1d = [25, 50, 65, 75, 125, 165, 225, 
                  325, 425, 525, 625, 725, 825, 925]
meta_model_inf_1d = []
for i in centers_inf_1d: 
    m = load_model('nets/inference_nets/1d/inference_1D_'+str(i)+'.h5')
    
    meta_model_inf_1d.append(m)
    

Using the net trained on trajectories of length 165

In [4]:
#choosing the net
net = meta_model_inf_1d[5]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X1[0],dim=1,task=1)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=1)
#prediction on trajectories of length 200 using a net trained on traj of length 165
pred_200_u165 = net.predict(data_rs)
print('predicted exponents', pred_200_u165.flatten())
print('ground truth', Y1[0])

predicted exponents [1.6737214 1.2866617 0.9582677]
ground truth [1.85, 1.25, 1.55]


Using the net trained on trajectories of length 225

In [5]:
#choosing the net
net = meta_model_inf_1d[6]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X1[0],dim=1,task=1)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=1)
#prediction on trajectories of length 200 using a net trained on traj of length 225
pred_200_u225 = net.predict(data_rs)
print('predicted exponents', pred_200_u225.flatten())
print('ground truth', Y1[0])

predicted exponents [1.6827123 1.2324138 0.9565854]
ground truth [1.85, 1.25, 1.55]


Using the combination of nearest nets, which in this case is 165 and 225 

In [6]:
pred_200_comb = many_net_uhd(nets = meta_model_inf_1d, traj_set = X1[0], centers = centers_inf_1d ,dim = 1, task =1)
print('predicted exponents',pred_200_comb)
print('ground truth', Y1[0])

predicted exponents [1.678966   1.255017   0.95728636]
ground truth [1.85, 1.25, 1.55]


# Classification in 2d

The output of the classiciation networks is an array giving the probability that the trajectory belongs to a model class. The classes, as in AnDi are [attm,ctrw,fbm,lw,sbm]

In [7]:
centers_class_2d = [25, 65, 125, 225, 425]
meta_model_class_2d = []
for i in centers_class_2d: 
    m = load_model('nets/classification_nets/2d/classification_2D_'+str(i)+'.h5')
    
    meta_model_class_2d.append(m)
    

In [8]:
#choosing the net
net = meta_model_class_2d[2]
#finding out the block size used by the chosen net
bs = net.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X2[1],dim=2,task=2)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=2)
#prediction on trajectories of length 200 using a net trained on traj of length 165
cla_200_u125 = net.predict(data_rs)
print("probability of each model class",'\n',cla_200_u125)
print("predicte most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cla_200_u125[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y2[1][i])])

probability of each model class 
 [[1.5193230e-09 1.0234864e-09 2.5234040e-06 9.9999750e-01 4.0343169e-09]
 [9.9036330e-01 4.4018007e-03 4.0434506e-06 5.2025188e-03 2.8314100e-05]
 [7.2885854e-03 1.8900841e-05 9.9123013e-01 7.8297366e-05 1.3840646e-03]]
predicte most likely model
lw
attm
fbm
Ground truth
lw
attm
fbm


Combining the nets

In [9]:
cla_200_comb = many_net_uhd(nets = meta_model_class_2d, traj_set = X2[1], centers = centers_class_2d ,dim = 2, task =2)
cla_200_comb = cla_200_comb.reshape(-1,5)
print("probability of each model class",'\n',cla_200_comb)
print("most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cla_200_comb[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y2[1][i])])    

probability of each model class 
 [[4.0953418e-10 3.5874057e-10 7.7262825e-07 9.9999917e-01 1.0397374e-09]
 [9.9664122e-01 1.7458976e-03 1.3668625e-05 1.5421249e-03 5.7050122e-05]
 [5.1056454e-03 3.8738741e-05 9.9334824e-01 2.4466257e-04 1.2627979e-03]]
most likely model
lw
attm
fbm
Ground truth
lw
attm
fbm


# Segmentation in 3d

Network to infer the exponents of the two segments and the switching point

In [8]:
model_a_t3d = tf.keras.models.load_model('nets/segmentation_nets/3d/T33D_inf.h5')


Network to classify the model of the second segment

In [14]:
class2_3d = tf.keras.models.load_model('nets/segmentation_nets/3d/T33D_c2.h5')

The output is [a1,a2,sin(2pi*t/T),cos(2pi*t/T)]

In [9]:

#finding out the block size used by the chosen net
bs = model_a_t3d.layers[0].input_shape[-1]

#normalizing the data
data = data_norm(X3[2],dim=3,task=3)


#reshaping the data

data_rs = data_reshape(data,bs=bs,dim=3)
#prediction on trajectories of length 200 using a net trained on traj of length 165
inf_seg = model_a_t3d.predict(data_rs)
pred_t = my_atan(inf_seg[:,2],inf_seg[:,3])*200/(2*np.pi)
cl2_3d = class2_3d.predict(data_rs)

print('predicted first exponents', inf_seg[:,0])
print('ground truth first exponents', Y3[2][:,3])
print('predicted second exponents', inf_seg[:,1])
print('ground truth second exponents', Y3[2][:,5])
print('predicted time and exponents', pred_t)
print('ground truth time', Y3[2][:,1])

predicted first exponents [0.57030606 1.1435541  0.5654856 ]
ground truth first exponents [1.25 1.7  0.25]
predicted second exponents [1.2434977 1.4401271 1.4441462]
ground truth second exponents [0.35 0.85 1.55]
predicted time and exponents [195.83064288 138.5685842    4.47446404]
ground truth time [184. 146.   1.]


In [13]:

print("probability of each model class for second segment",'\n',cl2_3d)
print("most likely model")
models = ['attm', 'ctrw', 'fbm', 'lw', 'sbm'] 
for i in range(len(data)):
    print(models[np.argmax(cl2_3d[i])])
print('Ground truth')    
for i in range(len(data)):
    print(models[int(Y3[2][i,2])])    

probability of each model class for second segment 
 [[7.2716689e-03 4.0842197e-04 1.9752480e-01 4.7767358e-03 7.9001844e-01]
 [4.9542618e-04 3.6407507e-04 1.6535368e-03 9.9664766e-01 8.3928369e-04]
 [2.2605374e-02 5.0307461e-03 1.7971978e-01 1.1503365e-02 7.8114086e-01]]
most likely model
sbm
lw
sbm
Ground truth
sbm
lw
attm
